In [1]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gym #pip install box2d box2d-kengz --user #openAI

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
class Memory:#训练数据收集
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []#是否终止
    
    def clear_memory(self):
        del self.actions[:] # [:] Deletes all the elements in the array
        del self.states[:]  
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

In [23]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor 输入一个state 决定一个action
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),#state是一个8维度向量
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),#action_dim=4，上下左右挑一个
                nn.Softmax(dim=-1) #dim=-1 得到四个概率值
                )
        
        # critic 级别越高打越厉害的怪，actor 做完了一个动作，到底可取不可取
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)#PPO2策略,最后返回一个value，用这个value state确定这个动作可取还是不可取（R-V）
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device)#游戏环境拿到手的，送入gpu
        action_probs = self.action_layer(state)##就是那个actor 得到四个概率值
        dist = Categorical(action_probs)#按照给定的概率分布来进行采样
        #作用是创建以参数probs为标准的类别分布，样本是来自 “0 … K-1” 的整数，其中 K 是probs参数的长度。
        #也就是说，按照传入的probs中给定的概率，在
        #相应的位置处进行取样，取样返回的是该位置的整数索引。
        action = dist.sample()# 根据现有分布抽样
        
        memory.states.append(state)#用list存起来
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
         

In [24]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)#构建两个网络模型
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)#实际与环境交互取参数的狸猫
        self.policy_old.load_state_dict(self.policy.state_dict())#在第一局一模一样，初始化时候一模一样
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        #对后续的印象打折
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)#gamma就是折扣，0.99很小
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()#2000个
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):#用以上的数据更新K_epochs 四次
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)#过去的acttion 走一遍
            
            # Finding the ratio (pi_theta / pi_theta__old):
            #由于是对数改成减法
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:就是ppo2公式的实现
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy#td差分先不管了
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())

In [25]:
def main():
    ############## Hyperparameters ##############
    env_name = "LunarLander-v2" ##游戏名称
    # creating environment
    env = gym.make(env_name)## 导入游戏
    state_dim = env.observation_space.shape[0]# 走一个action 得到一个state,这个state是一个8维度向量
    action_dim = 4 #上下左右
    render = True              # 是不是显示游戏过程
    solved_reward = 230         # stop training if avg_reward > solved_reward 
    log_interval = 20           # print avg reward in the interval
    max_episodes = 50000        # max training episodes
    max_timesteps = 300         # max timesteps in one episode
    n_latent_var = 64           # number of variables in hidden layer
    update_timestep = 2000      # update policy every n timesteps 多少个timestep之后供当前theta学习
    lr = 0.002
    betas = (0.9, 0.999)        #一阶二阶中心距更新权重参数
    gamma = 0.99                # discount factor
    K_epochs = 4                # update policy for K epochs 用狸猫取来的数据现在自己学多少次
    eps_clip = 0.2              # clip parameter for PPO
    random_seed = None
    #############################################
    
    if random_seed:
        torch.manual_seed(random_seed)
        env.seed(random_seed)
    
    memory = Memory()#训练数据收集，其实是玩了很多次游戏然后收集了才开始用
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
    #print(lr,betas)
    
    # logging variables
    running_reward = 0
    avg_length = 0
    timestep = 0
    
    # training loop
    for i_episode in range(1, max_episodes+1):#最多游戏次数
        state = env.reset()#初始化（重新玩）
        for t in range(max_timesteps):#避免无限循环，三百个行为就完事
            timestep += 1
            
            # Running policy_old:
            action = ppo.policy_old.act(state, memory)## 用police old（狸猫）产生数据
            state, reward, done, _ = env.step(action)#得到（新的状态，奖励，是否终止，额外的调试信息）
            
            # Saving reward and is_terminal:
            memory.rewards.append(reward)
            memory.is_terminals.append(done)
            
            # update if its time
            if timestep % update_timestep == 0:
                ppo.update(memory)#2000个更新一次数据
                memory.clear_memory()
                timestep = 0
            
            running_reward += reward
            if render:
                env.render()
            if done:
                break
                
        avg_length += t
        
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
            break
            
        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))
            
            print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
            running_reward = 0
            avg_length = 0

In [28]:
if __name__ == '__main__':
    main()

Episode 20 	 avg length: 89 	 reward: -184
Episode 40 	 avg length: 91 	 reward: -193
Episode 60 	 avg length: 105 	 reward: -213
Episode 80 	 avg length: 103 	 reward: -165
Episode 100 	 avg length: 97 	 reward: -117
Episode 120 	 avg length: 96 	 reward: -133
Episode 140 	 avg length: 89 	 reward: -137
Episode 160 	 avg length: 90 	 reward: -166
Episode 180 	 avg length: 95 	 reward: -203
Episode 200 	 avg length: 88 	 reward: -174
Episode 220 	 avg length: 84 	 reward: -122
Episode 240 	 avg length: 100 	 reward: -130
Episode 260 	 avg length: 91 	 reward: -125
Episode 280 	 avg length: 85 	 reward: -138
Episode 300 	 avg length: 88 	 reward: -143
Episode 320 	 avg length: 92 	 reward: -120
Episode 340 	 avg length: 99 	 reward: -127
Episode 360 	 avg length: 92 	 reward: -125
Episode 380 	 avg length: 87 	 reward: -116
Episode 400 	 avg length: 79 	 reward: -103
Episode 420 	 avg length: 79 	 reward: -101
Episode 440 	 avg length: 91 	 reward: -103
Episode 460 	 avg length: 93 	 re

In [2]:
 state_dim = env.observation_space.shape[0]

NameError: name 'env' is not defined